In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep
import os
option = webdriver.ChromeOptions()
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

# INIT

# visit web

In [6]:
import re
"""
ต่อจากด้านบนส่วนนี้จะเป็นการใช้ regular expression ในการสกัด link ออกมา (ขั้นตอนนี้จะไม่ได้ใช้ soup)
r'<a href="(https://www.britannica.com/place/(.*?)) ==> จาก pattern คือการสกัด 
pattern (https://www.britannica.com/place/(กี่ตัวอักษรก็ได้ และมีได้แค่ชุดเดียว) ปล.ลองลบ ? เพื่อความเข้าใจ
"""
url = "https://www.britannica.com/topic/list-of-countries-1993160"
html_data = requests.get(url).text
#name = re.findall(r'<a href="https://www.britannica.com/place/(.*?)" ',str(html_data))
regx_link = re.findall(r'<a href="(https://www.britannica.com/place/(.*?))" ',str(html_data))
links = []
for ele in regx_link:
    link  = ele[0]
    links.append(link)
    

In [7]:
name_of_country = []
info = []
list_flag = []
list_capital = []

for url in links:
    #driver.get(url)
    #soup = BeautifulSoup(driver.page_source ,"html.parser")
    r = requests.get(url)
    req = r.text
   
    ## GET NAME OF COUNTRY
    soup = BeautifulSoup(req ,"html.parser")
    nameOfCountry = soup.select("h1.mb-0")
    if (nameOfCountry ) :  
        name = nameOfCountry[0].get_text()
        name_of_country.append(name)
        
    ## GET FLAG URL
        linkFlag = re.findall('<img loading="lazy" src="(.*?lag.*?)"',req) #flag 2 แบบ Flag flag ทำให้ไม่ matched เลยใช้แค่ (_lag)
        if(linkFlag):
            list_flag.append(linkFlag[0])
        else:
            linkFlag = re.findall('<img loading="lazy" src="(.*?FLAG.*?)"',req)
            if(linkFlag): 
                    list_flag.append(linkFlag[0])
            else:
                linkFlag = re.findall('<img loading="lazy" src="(.*?)"',req) #ใช้กับประเทศ South Sudan เพราะใน link ไม่มีคำว่า flag เลย 
                list_flag.append(linkFlag[1])
    ## GET CAPTITAL
        nameOfCapital = re.findall('<a href="/place/.*?>(.*?)</a>',req)
        if(nameOfCapital):
            list_capital.append(nameOfCapital[0])
        else:
            nameOfCapital = re.findall('<dt class="d-inline">Capital:</dt>\n\t\t\t\t<dd class="d-inline">\n\t\t\t\t\t<span class="fact-item">(.*?)<',req)
            #list_capital.append(nameOfCapital[1])
            if(nameOfCapital):
                  list_capital.append(nameOfCapital[0])
            else:
                  list_capital.append("NAN")
    else:
        nameList.append("NAN")
        

# PACKING TO DATAFRAME


In [19]:
name_of_country = np.array(name_of_country)
list_flag = np.array(list_flag)
list_capital = np.array(list_capital)

name_df = pd.DataFrame(name_of_country ,columns=["name"])
capital_df = pd.DataFrame(list_capital ,columns=["capital"])
flagUrl_df = pd.DataFrame(list_flag ,columns=["flagUrl"])

pack = [name_df ,capital_df ,flagUrl_df]

country_data = pd.concat(pack, axis=1, join="inner")

In [20]:
country_data

,name,capital,flagUrl
0,Afghanistan,Kabul,https://cdn.britannica.com/40/5340-050-AA46700...
1,Albania,Tirana,https://cdn.britannica.com/00/6200-050-2E25CE9...
2,Algeria,Algiers,https://cdn.britannica.com/34/3034-050-077DE27...
3,Andorra,Andorra la Vella,https://cdn.britannica.com/83/5583-050-2F48FD3...
4,Angola,Luanda,https://cdn.britannica.com/12/4212-050-FC69793...
...,...,...,...
191,Venezuela,Caracas,https://cdn.britannica.com/04/4904-050-4D71EA8...
192,Vietnam,Hanoi,https://cdn.britannica.com/41/4041-050-BB4577F...
193,Yemen,Sanaa,https://cdn.britannica.com/85/5785-050-B3B0056...
194,Zambia,Lusaka,https://cdn.britannica.com/31/4231-050-B24F289...


# PACK TO JSON

In [57]:
"""
to_json(orient='records') จะ return เป็นแค่ String
การที่จะทำให้เป็น json files จำเป็นต้อง parsed โดยการใช้ json.loads("...json string fomat...")
"""
import json
json_str = country_data.to_json(orient='records')
parsed = json.loads(json_str)
parsed[:5]

[{'name': 'Afghanistan',
  'capital': 'Kabul',
  'flagUrl': 'https://cdn.britannica.com/40/5340-050-AA46700D/Flag-Afghanistan.jpg?w=400&h=235&c=crop'},
 {'name': 'Albania',
  'capital': 'Tirana',
  'flagUrl': 'https://cdn.britannica.com/00/6200-050-2E25CE97/Flag-Albania.jpg?w=400&h=235&c=crop'},
 {'name': 'Algeria',
  'capital': 'Algiers',
  'flagUrl': 'https://cdn.britannica.com/34/3034-050-077DE27D/Flag-Algeria.jpg?w=400&h=235&c=crop'},
 {'name': 'Andorra',
  'capital': 'Andorra la Vella',
  'flagUrl': 'https://cdn.britannica.com/83/5583-050-2F48FD32/Flag-Andorra.jpg?w=400&h=235&c=crop'},
 {'name': 'Angola',
  'capital': 'Luanda',
  'flagUrl': 'https://cdn.britannica.com/12/4212-050-FC697931/Flag-Angola.jpg?w=400&h=235&c=crop'}]